#### GPT4-o-mini

Основные гиперпараметры CatBoost для улучшения качества моделей оценки стоимости квартир (в порядке убывания важности, как я считаю, основываясь на опыте применения CatBoost):

1. learning_rate (или eta):
  •  Влияние: Регулирует величину шага при обновлении весов деревьев. Меньшие значения требуют большего количества итераций, но часто приводят к более точным и стабильным результатам.
  •  Подбор: Начните с значений в диапазоне [0.01, 0.1]. Используйте GridSearchCV или RandomizedSearchCV для поиска оптимального значения в этом диапазоне. Важно помнить, что learning_rate тесно связан с n_estimators.
  •  Рекомендация: При уменьшении learning_rate обычно необходимо увеличивать n_estimators. Пример: learning_rate=0.01, n_estimators=1000 может быть лучше, чем learning_rate=0.1, n_estimators=100.

2. n_estimators (или iterations):
  •  Влияние: Определяет количество деревьев в ансамбле. Слишком мало деревьев - недообучение, слишком много - переобучение (особенно без регуляризации).
  •  Подбор: Используйте early stopping (early_stopping_rounds) вместе с большим начальным значением n_estimators (например, 1000 или 2000). Early stopping остановит обучение, когда качество на валидационном наборе перестанет улучшаться.
  •  Рекомендация: Всегда используйте early stopping! Это критически важно для предотвращения переобучения и выбора оптимального количества деревьев. early_stopping_rounds обычно устанавливают в значениях от 50 до 200.

3. max_depth:
  •  Влияние: Ограничивает глубину каждого дерева. Слишком большая глубина приводит к переобучению, слишком маленькая - к недообучению.
  •  Подбор: Начните с диапазона [4, 10]. Оптимальное значение зависит от сложности ваших данных.
  •  Рекомендация: Для данных о недвижимости, где есть сложные зависимости, max_depth в районе 6-8 часто показывает хорошие результаты.

4. l2_leaf_reg (или reg_lambda):
  •  Влияние: L2 регуляризация. Добавляет штраф к функции потерь за большие значения весов листьев деревьев. Предотвращает переобучение.
  •  Подбор: Начните с диапазона [1, 10].
  •  Рекомендация: Это один из самых важных параметров регуляризации. Не пренебрегайте им.

5. loss_function:
  •  Влияние: Определяет функцию потерь, которую нужно минимизировать.
  •  Подбор: Для задач регрессии (оценка стоимости) наиболее часто используются:
    *  'RMSE': Root Mean Squared Error (корень из среднеквадратичной ошибки) - чувствителен к выбросам.
    *  'MAE': Mean Absolute Error (средняя абсолютная ошибка) - более устойчив к выбросам.
    *  'Huber': Комбинация RMSE и MAE, менее чувствителен к выбросам, чем RMSE.
    *  'Poisson': Подходит, если целевая переменная имеет пуассоновское распределение (например, количество продаж).
  •  Рекомендация: Начните с RMSE или MAE. Если в данных много выбросов, попробуйте Huber.

6. random_strength:
  •  Влияние: Добавляет случайность при выборе разбиений в деревьях. Увеличивает устойчивость модели к переобучению.
  •  Подбор: Обычно в диапазоне [0.1, 2].
  •  Рекомендация: Помогает, когда модель сильно переобучается.

7. border_count:
  •  Влияние: Определяет количество бинарных разбиений для каждого признака. Большее значение позволяет находить более точные разбиени
я, но увеличивает время обучения.
  •  Подбор: Обычно 32, 64, 128, 254.
  •  Рекомендация: Не всегда необходимо менять. Увеличение может помочь, если у вас много числовых признаков.

8. feature_fraction (или colsample_bylevel):
  •  Влияние: Определяет долю признаков, которые используются при построении каждого дерева. Помогает бороться с переобучением и улучшает обобщающую способность.
  •  Подбор: Диапазон [0.6, 0.9].
  •  Рекомендация: Используйте, если у вас очень много признаков.

9. task_type:
  * Влияние: Определяет, будет ли CatBoost использовать GPU или CPU.
  * Подбор: Если у вас есть GPU, установите task_type='GPU' для значительного ускорения обучения.
  * Рекомендация: Обязательно используйте GPU, если он доступен.

#### GPT4

### **Основные гиперпараметры, которые стоит учитывать:**  

1. **`max_depth`** – Определяет глубину деревьев.  
   - Чем больше, тем сложнее модель (больше переобучение).  
   - Оптимальный диапазон: **6–10**.  

2. **`learning_rate`** – Скорость обучения модели.  
   - Маленькие значения улучшают стабильность, но требуют больше итераций.  
   - Оптимальный диапазон: **0.01–0.1**.  

3. **`n_estimators`** – Количество деревьев (итераций).  
   - Чем выше, тем лучше модель подстраивается под данные.  
   - Обычно балансируют с `learning_rate`: **500–5000** (чем меньше `learning_rate`, тем больше `n_estimators`).  

4. **`early_stopping_rounds`** – Прерывает обучение, если качество ухудшается.  
   - Ты правильно ставишь **10**, но если данных много, можно увеличить до **20–50**.  

5. **`l2_leaf_reg`** – Регуляризация L2 для контроля переобучения.  
   - Оптимальный диапазон: **1–10** (по умолчанию **3**).  

6. **`depth`** – Альтернативный вариант `max_depth`, влияет на сложность деревьев.  
   - Обычно совпадает с `max_depth`, но можно тестировать **6–10**.  

7. **`boosting_type`** – Алгоритм построения бустинга.  
   - **Plain** – стандартный бустинг (по умолчанию).  
   - **Ordered** – может быть полезен, если данных мало.  

8. **`bagging_temperature`** – Контроль случайности в выборе объектов.  
   - **0–1**: при **1** больше случайности, при **0** модель стабильнее.  
   - Для твоей задачи можно тестировать **0.5–1**.  

9. **`random_strength`** – Контроль случайности разбиения деревьев.  
   - Оптимальный диапазон: **1–10** (обычно **1**).  

10. **`custom_loss`** – Можно использовать `['MAE', 'MAPE']`, так как ты их оцениваешь.  

### **Дополнительные параметры, если данных много:**  

- **`grow_policy`** – Как строятся деревья:  
  - `SymmetricTree` (по умолчанию) – сбалансированное дерево.  
  - `Depthwise` – полезно при глубине >10.  
  - `Lossguide` – лучше при больших данных.  

- **`colsample_bylevel`** – Доля признаков при построении дерева (0.5–1).  

- **`boost_from_average`** – Ускоряет обучение, можно **True**.  

### **Твой текущий процесс – хороший старт, но можно улучшить:**  
- Добавь подбор `l2_leaf_reg`, `bagging_temperature`, `random_strength`.  
- Проверь `boosting_type='Ordered'`, если данных мало.  
- `custom_loss=['MAE', 'MAPE']` для лучшего контроля качества.  

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

# Определяем модель CatBoost (без параметров, они будут подбираться)
model = CatBoostRegressor(loss_function='MAE', verbose=0, random_state=42)

# Сетка гиперпараметров
param_grid = {
    'max_depth': [6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 7],
    'bagging_temperature': [0, 0.5, 1]
}

# Запускаем GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Выводим лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)
print("Лучший MAE:", -grid_search.best_score_)

# Финальная модель с лучшими параметрами
best_model = grid_search.best_estimator_


In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Базовая модель CatBoost
model = CatBoostRegressor(loss_function='MAE', verbose=0, random_state=42)

# Определяем диапазоны гиперпараметров
param_dist = {
    'max_depth': np.arange(4, 11, 1),  # От 4 до 10
    'learning_rate': np.linspace(0.01, 0.3, 10),  # 10 значений от 0.01 до 0.3
    'l2_leaf_reg': np.linspace(1, 10, 10),  # 10 значений от 1 до 10
    'bagging_temperature': np.linspace(0, 1, 5),  # 5 значений от 0 до 1
    'iterations': np.arange(500, 5000, 500)  # От 500 до 5000 с шагом 500
}

# RandomizedSearchCV - тестируем 30 случайных комбинаций (n_iter=30)
random_search = RandomizedSearchCV(
    model, param_distributions=param_dist, n_iter=30, 
    cv=5, scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42
)

# Обучаем подбор параметров
random_search.fit(X_train, y_train)

# Выводим лучшие параметры
print("Лучшие параметры:", random_search.best_params_)
print("Лучший MAE:", -random_search.best_score_)

# Финальная модель с лучшими параметрами
best_model = random_search.best_estimator_
